<a href="https://colab.research.google.com/github/eceirem/Hukuki-Asistan/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr-tur
!pip install pytesseract pdf2image Pillow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 1s (148 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-tur
0 upgraded, 1 newly in

In [ ]:
import os
from google.colab import drive

# 1. Drive'ı Sıfırdan Bağla (Force Remount önemli)
drive.mount('/content/drive', force_remount=True)

# 2. Hedef Yol
# Kısayolu "Drive'ım" içine attıysan yol burasıdır:
target_path = "/content/drive/MyDrive/Legal NLP/Dataset/WhatsApp"

print(f"📂 Hedef Klasör Taranıyor: {target_path}")

if os.path.exists(target_path):
    all_files = os.listdir(target_path)

    # PDF Dosyalarını Filtrele
    pdf_files = [f for f in all_files if f.lower().endswith('.pdf')]

    print(f"📄 Toplam Dosya Sayısı: {len(all_files)}")
    print(f"✅ Bulunan PDF Sayısı: {len(pdf_files)}")

    if len(pdf_files) > 0:
        print("\n--- İlk 3 PDF Dosyası ---")
        for f in pdf_files[:3]:
            print(f" -> {f}")
        print("\nARTIK ANA KODU ÇALIŞTIRABİLİRSİN! 🚀")
    else:
        print("\n❌ HATA: Klasör bulundu ama içinde hala PDF yok.")
        print("Lütfen 'Benimle Paylaşılanlar'daki DOLU klasöre kısayol oluşturduğundan emin ol.")
        print("Mevcut içerik:", all_files)
else:
    print(f"❌ HATA: Yol bulunamadı: {target_path}")
    print("Lütfen Kısayolu 'Drive'ım' ana dizinine eklediğinden emin ol.")

Mounted at /content/drive
📂 Hedef Klasör Taranıyor: /content/drive/MyDrive/Legal NLP/Dataset/WhatsApp
📄 Toplam Dosya Sayısı: 302
✅ Bulunan PDF Sayısı: 302

--- İlk 3 PDF Dosyası ---
 -> 2020_327.pdf
 -> 2024_311.pdf
 -> 2021_1827.pdf

ARTIK ANA KODU ÇALIŞTIRABİLİRSİN! 🚀


In [ ]:
import pytesseract
from pdf2image import convert_from_path
import json
import re
import os
import random
from google.colab import drive

# --- AYARLAR ---
pytesseract.pytesseract.tesseract_cmd = 'tesseract'
POPPLER_PATH = None

class LegalDocProcessor:
    def __init__(self):
        os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata/'

        # --- REGEX KALIPLARI ---
        self.patterns = {
            # 1. Meta Veri Regexleri

            # ESAS NO: DOSYA, ESAS, E. gibi kısaltmaların arasını [.\s'’]* ile esneterek yakalar.
            "esas_no": r"(?:DOSYA|ESAS|E)[.\s'’]*NO[.\s'’]*:?\s*(\d{4}\s*/\s*\d+)",
            "karar_no": r"(?:KARAR|K)[.\s'’]*NO[.\s'’]*:?\s*(\d{4}\s*/\s*\d+)",

            # Dava Konusu (Davanın Konusu: veya DAVA: başlıklarını yakalar)
            "dava_konusu": r"(?:DAVANIN\s+KONUSU|DAVA)\s*[:;]\s*(.*?)(?:\n|DAVA TARİHİ|KARAR TARİHİ|GEREKÇELİ)",

            "dava_tarihi": r"DAVA\s+TARİHİ\s*[:;]?\s*(\d{1,2}[./-]\d{1,2}[./-]\d{4})",
            "karar_tarihi": r"KARAR\s+TARİHİ\s*[:;]?\s*(\d{1,2}[./-]\d{1,2}[./-]\d{4})",
            "mahkeme": r"T\.?C\.?\s*\n+.*?(MAHKEMESİ|HAKİMLİĞİ|DAİRESİ)",

            # 2. Bölümleme Regexleri
            # GEREĞİ DÜŞÜNÜLDÜ'den hemen sonra Dava konusu veya Davacı vekili... kısmı başlar.
            "facts_start": r"(GEREĞİ\s*DÜŞÜNÜLDÜ|GEREĞİ|TALEP)\s*[:;]?",
            "reasoning_start": r"(?<!GEREKÇELİ\s)(GEREKÇE|DELİLLERİN\s*DEĞERLENDİRİLMESİ|TÜM\s*DOSYA\s*KAPSAMI)\s*[:;]?",
            "verdict_start": r"(HÜKÜM|SONUÇ)\s*[:;]"
        }

    def clean_page_text(self, text):
        """
        Problem 1 Çözümü: Tesseract çıktılarını temizler. Sayfa başlıklarını ve gereksiz header/footer kısımlarını temizleyerek akışı düzeltir.
        """
        lines = text.split('\n')
        cleaned_lines = []
        for line in lines:
            stripped_line = line.strip()
            # 1. Tek başına duran sayfa numaralarını veya çok kısa satırları atla
            if len(stripped_line) < 3 and stripped_line.isdigit():
                continue

            # 2. T.C. Mahkeme başlıklarını (genellikle sayfa başında tekrar eden) atla
            if "T.C." in stripped_line and ("MAHKEMESİ" in stripped_line or "DAİRESİ" in stripped_line):
                 continue

            # 3. YANLIŞ OKUNAN/KARIŞAN Karakterleri temizle (Örn: | | yerine boşluk)
            cleaned_lines.append(stripped_line.replace('|', ''))

        return " ".join(cleaned_lines) # Satırları birleştirirken tek boşluk bırak

    def ocr_pdf_to_text(self, pdf_path):
        print(f"İşleniyor: {pdf_path}...")
        try:
            # PSM 3: Sütunlu düzeni veya blokları daha iyi algılar.
            custom_config = r'--psm 3'
            images = convert_from_path(pdf_path, dpi=300, poppler_path=POPPLER_PATH)

            full_text = ""
            for i, img in enumerate(images):
                page_text = pytesseract.image_to_string(img, lang='tur', config=custom_config)

                # Her sayfayı temizleme fonksiyonundan geçir
                cleaned_page = self.clean_page_text(page_text)

                full_text += cleaned_page + " " # Sayfalar arasına sadece bir boşluk koy

            return full_text
        except Exception as e:
            print(f"OCR Hatası: {e}")
            return ""

    def extract_metadata(self, text):
        metadata = {}

        # 1. Esas No (Düzeltildi)
        match_esas = re.search(self.patterns["esas_no"], text, re.IGNORECASE)
        # Metnin içinden sayıyı/yılı al
        if match_esas:
            # Sadece sayı grubunu al ve boşlukları temizle
            esas_num = re.search(r'(\d{4}\s*/\s*\d+)', match_esas.group(0), re.IGNORECASE)
            metadata["esas_no"] = esas_num.group(1).replace(" ", "") if esas_num else "Belirtilmemis"
        else:
            metadata["esas_no"] = "Belirtilmemis"

        # 2. Karar No (Düzeltildi)
        match_karar = re.search(self.patterns["karar_no"], text, re.IGNORECASE)
        if match_karar:
            karar_num = re.search(r'(\d{4}\s*/\s*\d+)', match_karar.group(0), re.IGNORECASE)
            metadata["karar_no"] = karar_num.group(1).replace(" ", "") if karar_num else "Belirtilmemis"
        else:
            metadata["karar_no"] = "Belirtilmemis"

        # 3. YENİ: Davanın Konusu (DAVA: veya DAVANIN KONUSU: ifadelerini kapsar)
        match_konu = re.search(self.patterns["dava_konusu"], text, re.IGNORECASE | re.DOTALL)
        if match_konu:
            # 1. grubu al (Dava konusunun içeriği)
            konu_raw = match_konu.group(1).strip()
            # Başta DAVA: veya DAVANIN KONUSU: gibi başlıklar varsa (regex içinde kalmış olabilir) temizle
            konu_cleaned = re.sub(r'^(DAVA|DAVANIN KONUSU)\s*[:;]', '', konu_raw, flags=re.IGNORECASE).strip()
            metadata["case_subject"] = konu_cleaned
        else:
            metadata["case_subject"] = "Belirtilmemis"

        # 4. Mahkeme Adı
        match_mahkeme = re.search(self.patterns["mahkeme"], text[:1500], re.DOTALL | re.IGNORECASE)
        if match_mahkeme:
            raw_court = match_mahkeme.group(0).replace("\n", " ").replace("T.C.", "").strip()
            metadata["court_name"] = re.sub(r'\s+', ' ', raw_court)
        else:
            metadata["court_name"] = "Bilinmiyor"

        # Case ID
        if metadata["esas_no"] != "Belirtilmemis" and metadata["karar_no"] != "Belirtilmemis":
            metadata["case_id"] = f"{metadata['esas_no']}-{metadata['karar_no']}"
        else:
            metadata["case_id"] = "UNKNOWN"

        return metadata

    def segmentation_rrl(self, text):
        clean_text = re.sub(r'\s+', ' ', text)
        segments = {"facts_text": "", "reasoning_text": "", "verdict_text": ""}

        reasoning_match = re.search(self.patterns['reasoning_start'], clean_text, re.IGNORECASE)
        verdict_match = re.search(self.patterns['verdict_start'], clean_text, re.IGNORECASE)

        reasoning_idx = reasoning_match.start() if reasoning_match else len(clean_text)
        verdict_idx = verdict_match.start() if verdict_match else len(clean_text)

        # Facts
        facts_match = re.search(self.patterns['facts_start'], clean_text, re.IGNORECASE)
        if facts_match:
            # GEREĞİ DÜŞÜNÜLDÜ'den Reasoning'e kadar al
            start_idx = facts_match.end()
            segments["facts_text"] = clean_text[start_idx:reasoning_idx].strip()
        else:
            # Fallback olarak metnin başından Reasoning'e kadar al (Meta verileri içerebilir)
            segments["facts_text"] = clean_text[:reasoning_idx].strip()

        # Reasoning
        if reasoning_match:
            end_idx = verdict_idx if verdict_idx > reasoning_idx else len(clean_text)
            segments["reasoning_text"] = clean_text[reasoning_match.end():end_idx].strip()

        # Verdict
        if verdict_match:
            segments["verdict_text"] = clean_text[verdict_match.end():].strip()

        return segments

    def extract_micro_features(self, text):
        features = {"mentioned_laws": []}

        # 1. Uzun format: "6100 sayılı ... Kanunu"
        laws_long = re.findall(r'(\d+\s+sayılı\s+[a-zA-ZğüşöçİĞÜŞÖÇ]+\s+Kanun[u]?)', text, re.IGNORECASE)

        # 2. Kısa format (HMK 341 vb.)
        laws_abbr_raw = re.findall(r'(HMK|TTK|TBK|İİK|CMK|MYK)\s*[.\'’]?\s*(?:\w+)?\s*(\d+)', text, re.IGNORECASE)
        formatted_abbr = [f"{item[0].upper()} {item[1]}" for item in laws_abbr_raw]

        features["mentioned_laws"] = list(set(laws_long + formatted_abbr))
        return features

    def process_file(self, pdf_path):
        raw_text = self.ocr_pdf_to_text(pdf_path)
        if not raw_text: return None

        return {
            "file_name": os.path.basename(pdf_path),
            "meta_data": self.extract_metadata(raw_text),
            "rrl_segments": self.segmentation_rrl(raw_text),
            "structural_features": self.extract_micro_features(raw_text)
        }

# --- MAIN ---
def main():
    drive.mount('/content/drive', force_remount=True)

    # KLASÖR YOLLARI
    base_dir = "/content/drive/MyDrive"
    input_folder = "Legal NLP/Dataset/WhatsApp"
    output_folder = "Legal NLP/Codes/myoutputs"

    input_dir = os.path.join(base_dir, input_folder)
    output_dir = os.path.join(base_dir, output_folder)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    if os.path.exists(input_dir):
        all_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.pdf')]

        if not all_files:
            print("Klasörde hiç PDF dosyası bulunamadı.")
            return

        selected_file = random.choice(all_files)
        print(f"\n--- Seçilen Dosya: {selected_file} ---")

        processor = LegalDocProcessor()
        result = processor.process_file(os.path.join(input_dir, selected_file))

        if result:
            out_name = f"processed_{os.path.splitext(selected_file)[0]}.json"
            out_path = os.path.join(output_dir, out_name)

            with open(out_path, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=4)

            print(f"BAŞARILI! Dosya kaydedildi: {out_path}")
            print("-" * 30)
            print("Esas No (Taranan):", result['meta_data']['esas_no'])
            print("Dava Konusu (Yakalanan):", result['meta_data']['case_subject'])
        else:
            print("Hata oluştu.")
    else:
        print(f"HATA: Klasör bulunamadı -> {input_dir}")

if __name__ == "__main__":
    main()

Mounted at /content/drive

--- Seçilen Dosya: 2024_295.pdf ---
İşleniyor: /content/drive/MyDrive/Legal NLP/Dataset/WhatsApp/2024_295.pdf...
BAŞARILI! Dosya kaydedildi: /content/drive/MyDrive/Legal NLP/Codes/myoutputs/processed_2024_295.json
------------------------------
Esas No (Taranan): 2024/295
Dava Konusu (Yakalanan): Menfi Tespit, İstirdat (Eser Sözleşmesinden Kaynaklanan) İSTEM : İhtiyati Tedbir


Tüm Dosyaları aynı yere alıyorum

In [ ]:
import os
import shutil
from google.colab import drive
import time

def consolidate_pdfs_to_single_folder():
    """
    Dataset altındaki tüm (alt klasörlerdeki veya kökteki) PDF'leri
    All_PDFs_Consolidated klasörüne taşır.
    """
    drive.mount('/content/drive', force_remount=True)

    # 📌 AYARLAR
    # Tarama başlangıç noktası: Tüm dağınık dosyaların bulunduğu kök klasör
    base_dir = "/content/drive/MyDrive/Legal NLP/Dataset"

    # Tüm PDF'lerin taşınacağı hedef klasör
    target_dir_name = "All_PDFs_Consolidated"
    target_dir = os.path.join(base_dir, target_dir_name)

    # Hedef klasör yoksa oluştur
    if not os.path.exists(target_dir):
        os.makedirs(target_dir, exist_ok=True)
        print(f"Hedef klasör oluşturuldu: {target_dir}")

    print(f"\n--- PDF TOPLAMA İŞLEMİ BAŞLADI ---")
    print(f"Hedef Klasör: {target_dir}")

    processed_count = 0

    # 📁 Klasörleri Yinelemeli Olarak Tarama (os.walk)
    for root, dirs, files in os.walk(base_dir):

        # Eğer root (mevcut klasör) zaten hedef klasör ise atla (kendi içini taşımasın)
        if root == target_dir:
            continue

        for filename in files:
            # Sadece PDF dosyalarını al
            if filename.lower().endswith('.pdf'):

                old_path = os.path.join(root, filename)
                base, ext = os.path.splitext(filename)

                # --- Çakışma Kontrolü ve Yeniden Adlandırma ---
                new_filename = filename
                new_path = os.path.join(target_dir, new_filename)
                counter = 1

                # Eğer hedef klasörde aynı isimde dosya varsa sonuna sayı ekle (dava (1).pdf gibi)
                while os.path.exists(new_path):
                    new_filename = f"{base}_{counter}{ext}"
                    new_path = os.path.join(target_dir, new_filename)
                    counter += 1

                try:
                    # Dosyayı taşı
                    shutil.move(old_path, new_path)
                    processed_count += 1
                    print(f"Taşındı: {old_path.replace(base_dir, '')} -> {new_filename}")
                except Exception as e:
                    print(f"❌ HATA ({filename}): {e}")

    print(f"\n=== İŞLEM TAMAMLANDI ===")
    print(f"Toplam {processed_count} adet PDF dosyası '{target_dir_name}' klasörüne taşındı.")

if __name__ == "__main__":
    consolidate_pdfs_to_single_folder()

Mounted at /content/drive

--- PDF TOPLAMA İŞLEMİ BAŞLADI ---
Hedef Klasör: /content/drive/MyDrive/Legal NLP/Dataset/All_PDFs_Consolidated
Taşındı: /2018_560.pdf -> 2018_560.pdf
Taşındı: /2021_966.pdf -> 2021_966.pdf
Taşındı: /2022_373.pdf -> 2022_373.pdf
Taşındı: /2022_2357 (1).pdf -> 2022_2357 (1).pdf
Taşındı: /2021_109.pdf -> 2021_109.pdf
Taşındı: /2021_1273.pdf -> 2021_1273.pdf
Taşındı: /2023_668.pdf -> 2023_668.pdf
Taşındı: /2024_2125.pdf -> 2024_2125.pdf
Taşındı: /2023_908.pdf -> 2023_908.pdf
Taşındı: /2023_288.pdf -> 2023_288.pdf
Taşındı: /2025_967.pdf -> 2025_967.pdf
Taşındı: /2020_314.pdf -> 2020_314.pdf
Taşındı: /2020_300.pdf -> 2020_300.pdf
Taşındı: /2022_3049.pdf -> 2022_3049.pdf
Taşındı: /2024_271.pdf -> 2024_271.pdf
Taşındı: /2023_2336.pdf -> 2023_2336.pdf
Taşındı: /2019_2020.pdf -> 2019_2020.pdf
Taşındı: /2020_155.pdf -> 2020_155.pdf
Taşındı: /2019_1579.pdf -> 2019_1579.pdf
Taşındı: /2020_217.pdf -> 2020_217.pdf
Taşındı: /2021_1675.pdf -> 2021_1675.pdf
Taşındı: /2021_126

Toplam dosya sayısı

In [ ]:
import os
from google.colab import drive

def count_files_in_dataset_folders():
    """
    Structured_Judgements ve All_PDFs_Consolidated klasörlerindeki
    dosya sayılarını ayrı ayrı hesaplar ve raporlar.
    """
    drive.mount('/content/drive', force_remount=True)

    # 📌 KÖK KLASÖR: Dataset'in bulunduğu yer
    base_dir = "/content/drive/MyDrive/Legal NLP/Dataset"

    # 📁 Hedef Klasörler
    folder_paths = {
        "PDF Dosyaları (Girdi)": os.path.join(base_dir, "All_PDFs_Consolidated"),
        "JSON Kararlar (Çıktı)": os.path.join(base_dir, "Structured_Judgements")
    }

    print("--- DOSYA SAYIM RAPORU BAŞLADI ---")

    total_count = 0

    for label, path in folder_paths.items():
        if os.path.exists(path):
            # Klasör içindeki tüm öğeleri listeler
            all_items = os.listdir(path)

            # Sadece dosyaları say (Klasörleri atla)
            file_count = sum(1 for item in all_items if os.path.isfile(os.path.join(path, item)))

            print(f"[{label}]: {file_count} adet dosya")
            print("---------------------------------")
            total_count += file_count
        else:
            print(f"⚠️ HATA: '{label}' klasörü ({path}) bulunamadı.")


if __name__ == "__main__":

    count_files_in_dataset_folders()

Mounted at /content/drive
--- DOSYA SAYIM RAPORU BAŞLADI ---
[PDF Dosyaları (Girdi)]: 1000 adet dosya
---------------------------------
[JSON Kararlar (Çıktı)]: 1000 adet dosya
---------------------------------


prompt yanlışlıkla değiştiği için mentioned laws kısmı silinmiş.
kaç tanesinde eksik olduğuna bakacağız

In [ ]:
import os
import json
from google.colab import drive

def count_jsons_missing_laws_before_deletion():
    """
    Structured_Judgements klasöründeki, mentioned_laws alanı eksik olan
    JSON dosyalarının sayısını hesaplar.
    """
    drive.mount('/content/drive', force_remount=True)

    # 📌 JSON KLASÖRÜ
    output_dir = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"

    missing_laws_count = 0
    checked_count = 0

    if not os.path.exists(output_dir):
        print(f"❌ HATA: Çıktı klasörü bulunamadı: {output_dir}")
        return

    print(f"--- SİLİNECEK DOSYA SAYISI KONTROLÜ BAŞLADI ---")

    for filename in os.listdir(output_dir):
        if filename.lower().endswith('.json'):
            file_path = os.path.join(output_dir, filename)

            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    checked_count += 1

                # 🔍 KONTROL: 'structural_features' bloğu ve içinde 'mentioned_laws' var mı?
                # VEYA 'mentioned_laws' varsa bile boş liste mi? (Opsiyonel Kontrol)
                if ("structural_features" not in data or
                    "mentioned_laws" not in data["structural_features"]):

                    missing_laws_count += 1

            except json.JSONDecodeError:
                # Bozuk JSON dosyalarını da sayıma dahil edelim, çünkü onlar da silinecek.
                missing_laws_count += 1

            except Exception as e:
                print(f"Hata: {filename} için bilinmeyen hata.")

    print("-------------------------------------------------------")
    print(f"Toplam JSON Dosyası Kontrol Edildi: {checked_count} adet")
    print(f"⚠️ Silinmeye Uygun Tahmini Dosya Sayısı: {missing_laws_count} adet")
    print("-------------------------------------------------------")

if __name__ == "__main__":
    count_jsons_missing_laws_before_deletion()

Mounted at /content/drive
--- SİLİNECEK DOSYA SAYISI KONTROLÜ BAŞLADI ---
-------------------------------------------------------
Toplam JSON Dosyası Kontrol Edildi: 985 adet
⚠️ Silinmeye Uygun Tahmini Dosya Sayısı: 32 adet
-------------------------------------------------------


silme kodu

In [ ]:
import os
import json
from google.colab import drive

def delete_jsons_missing_laws():
    """
    Structured_Judgements klasöründeki, structural_features / mentioned_laws
    alanı eksik olan JSON dosyalarını siler.
    """
    drive.mount('/content/drive', force_remount=True)

    # 📌 JSON KLASÖRÜ
    output_dir = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"

    deleted_count = 0
    checked_count = 0

    if not os.path.exists(output_dir):
        print(f"❌ HATA: Çıktı klasörü bulunamadı: {output_dir}")
        return

    print(f"--- EKSİK HUKUK MADDESİ OLAN JSON'LAR TEMİZLENİYOR ---")

    for filename in os.listdir(output_dir):
        if filename.lower().endswith('.json'):
            file_path = os.path.join(output_dir, filename)

            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    checked_count += 1

                # 🔍 KONTROL: 'structural_features' bloğu ve içinde 'mentioned_laws' var mı?
                if ("structural_features" not in data or
                    "mentioned_laws" not in data["structural_features"]):

                    # Eksikse sil
                    os.remove(file_path)
                    deleted_count += 1
                    print(f"🗑️ Silindi (Eksik Alan): {filename}")

            except json.JSONDecodeError:
                # Bozuk JSON dosyaları varsa sil
                os.remove(file_path)
                deleted_count += 1
                print(f"❌ Silindi (Bozuk JSON): {filename}")

            except Exception as e:
                print(f"Hata: {filename} için bilinmeyen hata: {e}")

    print("-------------------------------------------------------")
    print(f"Toplam Kontrol Edilen Dosya: {checked_count}")
    print(f"Toplam Silinen Eksik Dosya: {deleted_count}")
    print(f"Temizlik Tamamlandı. Kalan dosyalar yeniden işlenecek.")

if __name__ == "__main__":
    delete_jsons_missing_laws()

Mounted at /content/drive
--- EKSİK HUKUK MADDESİ OLAN JSON'LAR TEMİZLENİYOR ---
🗑️ Silindi (Eksik Alan): vision_llm_processed_2017_534.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2017_603.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2018_1895.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2018_227.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2018_38.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2019_1184.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2019_315.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2019_352.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2019_456.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2019_519.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2019_635.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2020_192.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2020_318.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2021_110.json
🗑️ Silindi (Eksik Alan): vision_llm_processed_2021_1175.json
🗑️ Silindi (Eksik

llm çıktılarında hatalar var
kimisi görsel yok diyor, kimisi token sınırına takılmış summary kısmı yok

In [ ]:
import os
import json
from google.colab import drive
import re

def count_jsons_missing_laws_before_deletion():
    """
    Structured_Judgements klasöründeki, eksik 'mentioned_laws' veya
    anlamsız içerik (null, görselde yok) içeren JSON'ları sayar.
    """
    drive.mount('/content/drive', force_remount=True)

    # 📌 JSON KLASÖRÜ
    output_dir = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"

    missing_laws_count = 0
    checked_count = 0

    # Anlamsız içerik tespiti için anahtar kelimeler
    EMPTY_KEYWORDS = ["belirtilmemiş", "null", "yer almamakta", "görselde dava bilgileri", "bilgi yok", "unknown", "verilmemiştir"]

    if not os.path.exists(output_dir):
        print(f"❌ HATA: Çıktı klasörü bulunamadı: {output_dir}")
        return

    print(f"--- SİLİNECEK DOSYA SAYISI KONTROLÜ BAŞLADI ---")

    for filename in os.listdir(output_dir):
        if filename.lower().endswith('.json'):
            file_path = os.path.join(output_dir, filename)

            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    checked_count += 1

                should_count = False

                # 1. YAPISAL KONTROL (mentioned_laws kayıp mı?)
                if ("structural_features" not in data or
                    "mentioned_laws" not in data["structural_features"]):
                    should_count = True

                # 2. İÇERİKSEL KONTROL (Anlamsız metin var mı?)
                if not should_count:
                    for key in ["facts_text", "reasoning_text", "verdict_text"]:
                        text = data.get("rrl_segments", {}).get(key, "")
                        if any(keyword in str(text).lower() for keyword in EMPTY_KEYWORDS):
                            should_count = True
                            break

                if should_count:
                    missing_laws_count += 1

            except json.JSONDecodeError:
                # Bozuk JSON dosyalarını da sayıma dahil et
                missing_laws_count += 1

            except Exception:
                pass # Bilinmeyen hataları yoksay

    print("-------------------------------------------------------")
    print(f"Toplam JSON Dosyası Kontrol Edildi: {checked_count} adet")
    print(f"⚠️ SİLİNMEYE UYGUN HATALI/EKSİK DOSYA SAYISI: {missing_laws_count} adet")
    print("-------------------------------------------------------")

if __name__ == "__main__":
    count_jsons_missing_laws_before_deletion()

Mounted at /content/drive
--- SİLİNECEK DOSYA SAYISI KONTROLÜ BAŞLADI ---
-------------------------------------------------------
Toplam JSON Dosyası Kontrol Edildi: 1004 adet
⚠️ SİLİNMEYE UYGUN HATALI/EKSİK DOSYA SAYISI: 9 adet
-------------------------------------------------------


38 dosya bozuk peki neden?

In [ ]:
import os
import json
from google.colab import drive
import re

def report_bad_jsons_reasons():
    """
    Structured_Judgements klasöründeki hatalı JSON dosyalarını ve hataların
    spesifik nedenlerini (Yapısal/İçeriksel/Bozuk Format) raporlar.
    """
    drive.mount('/content/drive', force_remount=True)

    # 📌 JSON KLASÖRÜ
    output_dir = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"

    # Anlamsız içerik tespiti için anahtar kelimeler
    EMPTY_KEYWORDS = ["Belirtilmemiş", "null", "görselde dava bilgileri veya iddialar yer almamaktadır", "unknown"]

    bad_files_report = []
    checked_count = 0

    if not os.path.exists(output_dir):
        print(f"❌ HATA: Çıktı klasörü bulunamadı: {output_dir}")
        return

    print(f"--- HATALI JSON NEDEN RAPORU BAŞLADI ---")

    for filename in os.listdir(output_dir):
        if filename.lower().endswith('.json'):
            file_path = os.path.join(output_dir, filename)

            reason = None

            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    checked_count += 1

                # 1. YAPISAL KONTROL (mentioned_laws kayıp mı?)
                # Bu, 7 Aralık'tan sonraki prompt revizyonundan kaynaklanan hatadır.
                if ("structural_features" not in data or
                    "mentioned_laws" not in data["structural_features"]):
                    reason = "Yapısal Eksiklik (mentioned_laws kayıp)"

                # 2. İÇERİKSEL KONTROL (Anlamsız metin var mı?)
                # Bu, LLM'in görseli okuyamaması veya token sınırına takılmasıdır.
                if not reason:
                    for key in ["facts_text", "reasoning_text", "verdict_text"]:
                        text = data.get("rrl_segments", {}).get(key, "")
                        if any(keyword in str(text).lower() for keyword in EMPTY_KEYWORDS):
                            reason = f"İçeriksel Eksiklik ({key} alanında anlamsız veri)"
                            break

            except json.JSONDecodeError:
                # LLM yarım bırakıp bozuk JSON üretti.
                reason = "Bozuk JSON Formatı"

            except Exception:
                reason = "Bilinmeyen JSON Yükleme Hatası"


            if reason:
                bad_files_report.append((filename, reason))


    # --- RAPOR SUNUMU ---
    print("\n=======================================================")
    print(f"Toplam Kontrol Edilen JSON Dosyası: {checked_count} adet")
    print(f"⚠️ Toplam Hatalı Dosya Sayısı: {len(bad_files_report)} adet")
    print("=======================================================\n")

    if bad_files_report:
        print("HATA NEDENLERİ LİSTESİ:")
        for filename, reason in bad_files_report:
            print(f"- {filename}: {reason}")
    else:
        print("Mükemmel! Hata kriterlerine uyan dosya bulunamadı.")

if __name__ == "__main__":
    report_bad_jsons_reasons()

Mounted at /content/drive
--- HATALI JSON NEDEN RAPORU BAŞLADI ---

Toplam Kontrol Edilen JSON Dosyası: 1002 adet
⚠️ Toplam Hatalı Dosya Sayısı: 2 adet

HATA NEDENLERİ LİSTESİ:
- vision_llm_processed_2019_456.json: Yapısal Eksiklik (mentioned_laws kayıp)
- vision_llm_processed_2022_1126.json: Yapısal Eksiklik (mentioned_laws kayıp)


kendini tekrarlayan bozuk veri var mı?

In [ ]:
import os
import json
from google.colab import drive
import re

def check_for_repetition(text: str, min_length: int = 500, chunk_size: int = 50) -> bool:
    """
    Metin içinde aynı blokların aşırı tekrarlanıp tekrarlanmadığını kontrol eder.
    (LLM'in token sınırında kilitlenme hatası)
    """
    text = text.strip()
    # Metin çok kısaysa kontrol etme
    if len(text) < min_length:
        return False

    # Metni 50'şer karakterlik bloklara ayır
    chunks = [text[i:i + chunk_size].strip() for i in range(0, len(text) - chunk_size, chunk_size)]

    if len(chunks) < 5:
        return False

    # İlk dört bloğun aynı olup olmadığını kontrol et (tekrarı yakalar)
    if chunks[1] == chunks[2] and chunks[2] == chunks[3]:
        return True

    return False

def count_repetitive_jsons_all_rrl():
    """
    Structured_Judgements klasöründeki, rrl_segments içindeki tüm alanlarda
    (facts, reasoning, verdict) aşırı tekrarlama hatası olan dosyaları sayar.
    """
    drive.mount('/content/drive', force_remount=True)

    # 📌 JSON KLASÖRÜ
    output_dir = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"

    repetitive_count = 0
    checked_count = 0

    RRL_KEYS = ["facts_text", "reasoning_text", "verdict_text"] # Kontrol edilecek alanlar

    if not os.path.exists(output_dir):
        print(f"❌ HATA: Çıktı klasörü bulunamadı: {output_dir}")
        return

    print(f"--- TÜM RRL SEGMENTLERİNDE TEKRAR SAYIM RAPORU BAŞLADI ---")

    for filename in os.listdir(output_dir):
        if filename.lower().endswith('.json'):
            file_path = os.path.join(output_dir, filename)

            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    checked_count += 1

                is_repetitive = False

                # Bütün RRL anahtarlarını kontrol et
                for key in RRL_KEYS:
                    text_to_check = data.get("rrl_segments", {}).get(key, "")

                    if check_for_repetition(text_to_check):
                        # Tek bir alanda bile tekrar varsa, o dosya hatalıdır.
                        is_repetitive = True
                        break

                if is_repetitive:
                    repetitive_count += 1

            except json.JSONDecodeError:
                # Bozuk JSON'ları yoksay (bu kod sadece tekrarlamayı bulmaya odaklanmıştır)
                pass

            except Exception:
                pass

    print("=======================================================")
    print(f"Toplam Kontrol Edilen JSON Dosyası: {checked_count} adet")
    print(f"⚠️ Toplam Tekrarlayan/Döngüye Girmiş Dosya Sayısı: {repetitive_count} adet")
    print("=======================================================")

if __name__ == "__main__":
    count_repetitive_jsons_all_rrl()

Mounted at /content/drive
--- TÜM RRL SEGMENTLERİNDE TEKRAR SAYIM RAPORU BAŞLADI ---
Toplam Kontrol Edilen JSON Dosyası: 1002 adet
⚠️ Toplam Tekrarlayan/Döngüye Girmiş Dosya Sayısı: 0 adet


pdf davalarda olup da jsona dönüşmemiş dosya adları

In [ ]:
import os
from google.colab import drive

def find_unprocessed_pdfs():
    """
    All_PDFs_Consolidated klasöründeki PDF'lerden hangilerinin
    Structured_Judgements klasöründe karşılığı olan JSON'a dönüşmediğini bulur.
    """
    drive.mount('/content/drive', force_remount=True)

    # 📌 KLASÖR YOLLARI
    input_dir = "/content/drive/MyDrive/Legal NLP/Dataset/All_PDFs_Consolidated"
    output_dir = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"

    if not os.path.exists(input_dir):
        print(f"❌ HATA: PDF klasörü bulunamadı: {input_dir}")
        return

    if not os.path.exists(output_dir):
        print(f"❌ HATA: JSON klasörü bulunamadı: {output_dir}")
        return

    # 1. Tüm PDF Dosyalarını Al
    all_pdfs = set()
    for f in os.listdir(input_dir):
        if f.lower().endswith('.pdf'):
            # Dosya uzantısını (.pdf) kaldırarak temel ismi al
            base_name = os.path.splitext(f)[0]
            all_pdfs.add(base_name)

    # 2. Tüm Mevcut JSON Dosyalarını Al
    processed_jsons = set()
    # JSON dosyalarının adları "vision_llm_processed_2022_1234.json" formatında.
    for f in os.listdir(output_dir):
        if f.lower().endswith('.json'):
            # "vision_llm_processed_" önekini ve ".json" uzantısını kaldır
            if f.startswith("vision_llm_processed_"):
                base_name = f[len("vision_llm_processed_"):-len(".json")]
                processed_jsons.add(base_name)

    # 3. Eksik Olanları Bul (PDFler - JSONlar)
    unprocessed_base_names = all_pdfs - processed_jsons

    # 4. Eksik Olan PDF'lerin Tam İsimlerini Oluştur
    unprocessed_pdfs = [f"{base_name}.pdf" for base_name in unprocessed_base_names]

    # --- RAPOR ---
    print("\n=======================================================")
    print(f"📂 Toplam PDF Dosyası (Girdi): {len(all_pdfs)} adet")
    print(f"📂 Toplam İşlenmiş JSON (Çıktı): {len(processed_jsons)} adet")
    print("-------------------------------------------------------")

    if unprocessed_pdfs:
        print(f"❌ JSON'a DÖNÜŞMEMİŞ PDF SAYISI: {len(unprocessed_pdfs)} adet")
        print("-------------------------------------------------------")
        print("EKSİK PDF DOSYA LİSTESİ:")

        # Listeyi alfabetik olarak sıralayarak daha kolay takip et
        for pdf_name in sorted(unprocessed_pdfs):
            print(f"- {pdf_name}")
    else:
        print("🎉 Tebrikler! Tüm PDF'ler başarıyla JSON'a dönüştürülmüş.")

if __name__ == "__main__":
    find_unprocessed_pdfs()

Mounted at /content/drive

📂 Toplam PDF Dosyası (Girdi): 1000 adet
📂 Toplam İşlenmiş JSON (Çıktı): 1000 adet
-------------------------------------------------------
❌ JSON'a DÖNÜŞMEMİŞ PDF SAYISI: 1 adet
-------------------------------------------------------
EKSİK PDF DOSYA LİSTESİ:
- 2020_1295 (1).pdf


json var pdf yok

In [ ]:
import os
from google.colab import drive

def find_jsons_without_pdfs():
    """
    Structured_Judgements klasöründeki JSON'lardan hangilerinin
    All_PDFs_Consolidated klasöründe karşılığı olan PDF'e sahip olmadığını bulur.
    """
    drive.mount('/content/drive', force_remount=True)

    # 📌 KLASÖR YOLLARI
    input_dir = "/content/drive/MyDrive/Legal NLP/Dataset/All_PDFs_Consolidated"
    output_dir = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"

    if not os.path.exists(input_dir):
        print(f"❌ HATA: PDF klasörü (Input) bulunamadı: {input_dir}")
        return

    if not os.path.exists(output_dir):
        print(f"❌ HATA: JSON klasörü (Output) bulunamadı: {output_dir}")
        return

    # 1. Tüm Mevcut PDF Dosyalarının Temel İsimlerini Al
    # (Örn: "2022_1234.pdf" -> "2022_1234")
    existing_pdf_base_names = set()
    for f in os.listdir(input_dir):
        if f.lower().endswith('.pdf'):
            base_name = os.path.splitext(f)[0]
            existing_pdf_base_names.add(base_name)

    # 2. Fazladan Olan JSON Dosyalarını Bul
    jsons_without_pdf = []

    for f in os.listdir(output_dir):
        if f.lower().endswith('.json'):

            # JSON dosyasının temel adını al (Örn: "vision_llm_processed_2022_1234.json" -> "2022_1234")
            if f.startswith("vision_llm_processed_"):
                # "vision_llm_processed_" önekini ve ".json" uzantısını kaldır
                json_base_name = f[len("vision_llm_processed_"):-len(".json")]

                # Çakışma sonucu oluşan kopyaları yoksay (Örn: ... (1).json)
                if "(1)" in json_base_name or "(2)" in json_base_name:
                    continue

                # Bu temel isim, PDF setinde var mı?
                if json_base_name not in existing_pdf_base_names:
                    jsons_without_pdf.append(f)
            else:
                 # Ön ekine uymayan JSON'ları da fazlalık say
                jsons_without_pdf.append(f)

    # --- RAPOR ---
    print("\n=======================================================")
    print(f"📂 Toplam PDF Dosyası (Input): {len(existing_pdf_base_names)} adet")
    print(f"📂 Toplam Fazla JSON Dosyası: {len(jsons_without_pdf)} adet")
    print("-------------------------------------------------------")

    if jsons_without_pdf:
        print("⚠️ KARŞILIĞI OLMAYAN JSON DOSYA LİSTESİ (Fazlalıklar):")

        # Listeyi alfabetik olarak sıralayarak daha kolay takip et
        for json_name in sorted(jsons_without_pdf):
            print(f"- {json_name}")

        print("\nİpuçları: Bu JSON'lara karşılık gelen PDF'ler silinmiş veya taşınmıştır.")
        print("Bu JSON'lar, silinebilir veya kurtarılacak PDF'ler bulunana kadar saklanabilir.")
    else:
        print("🎉 Tebrikler! JSON klasörünüzde karşılığı olmayan PDF bulunmamaktadır.")

if __name__ == "__main__":
    find_jsons_without_pdfs()

Mounted at /content/drive

📂 Toplam PDF Dosyası (Input): 1000 adet
📂 Toplam Fazla JSON Dosyası: 1 adet
-------------------------------------------------------
⚠️ KARŞILIĞI OLMAYAN JSON DOSYA LİSTESİ (Fazlalıklar):
- vision_llm_processed_2020_1295.json

İpuçları: Bu JSON'lara karşılık gelen PDF'ler silinmiş veya taşınmıştır.
Bu JSON'lar, silinebilir veya kurtarılacak PDF'ler bulunana kadar saklanabilir.
